In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import scipy
import scipy.stats
import scipy.integrate
from scipy.integrate import quad
from scipy.stats import uniform, norm
from jupyterthemes import jtplot
jtplot.style()

## Основные задания
1) По методу Монте-Карло вычислить приближенное значение определенного интеграла. Параметр числа итераций n выбрать большим 1000. Сравнить полученное значение либо с точным значением (если его получится вычислить), либо с приближенным, полученным в каком-либо математическом пакете (например, в mathematica).

### Параметры
(вариант 4)

---

а) $\int_0^{\pi} (x \mathrm{cos}(x))^2 dx$

б) $\int_{-\infty}^{\infty} \frac{dx}{(x^2+x+1)^2} $

в) $\iint\limits_{x^2+y^2<1} \, \mathrm{ln}\left(\frac{1}{\sqrt{x^2+y^2}}\right) dx dy $

---

$A=\begin{pmatrix}
0.75 & 0 & 0.25 \\
0.3 & 0.5 & 0.2 \\
0.2 & 0.2 & 0.6 
\end{pmatrix}\cdot 0.9 
,\quad
f=\begin{pmatrix}
0.33333 \\
0.33333 \\
0.33334
\end{pmatrix}
$

In [2]:
def compute_integral(g, x0, x1, distr, n=10000):
    eta = distr.rvs(n)
    approx = np.mean(g(eta)/distr.pdf(eta))
    exact,_ = quad(g, x0, x1)
    print(f'Approximate: {approx}, exact: {exact}')

def compute_integral_2d(g, n=1000000):
    distr_r2 = uniform()
    distr_phi = uniform(loc=-np.pi, scale=2*np.pi)
    length = np.sqrt(distr_r2.rvs(n))
    angle = distr_phi.rvs(n)
    eta_x = length * np.cos(angle)
    eta_y = length * np.sin(angle)
    # plt.scatter(eta_x, eta_y)
    # plt.show()
    pdf = lambda x,y: distr_r2.pdf(x**2+y**2)*distr_phi.pdf(np.arctan2(y,x))*2
    approx = np.mean(g(eta_x, eta_y)/pdf(eta_x, eta_y))
    exact = quad(lambda r: r*np.log(1/r), 0, 1)[0]*2*np.pi
    print(f'Approximate: {approx}, exact: {exact}')


print('a)')
g = lambda x: (x*np.cos(x))**2
x0, x1 = 0, np.pi
distr = uniform(x0,x1)
compute_integral(g, x0, x1, distr)

print('b)')
g = lambda x: 1/(x**2+x+1)**2
x0, x1 = -np.inf, np.inf
distr = norm()
compute_integral(g, x0, x1, distr)

print('c)')
g = lambda x,y: np.log(1/np.sqrt(x**2+y**2))
compute_integral_2d(g)

a)
Approximate: 6.077193213198557, exact: 5.953110943447418
b)
Approximate: 2.4033286636220335, exact: 2.4183991523122903
c)
Approximate: 1.572756625701668, exact: 1.5707963267948966


## Дополнительные задания
1) Решить систему линейных алгебраических уравнений $x=Ax+f$ методом Монте-Карло. Сравнить с решением данного уравнения, полученным произвольным численным методом или решением в произвольном математическом пакете. В качестве матрицы $A$ взять матрицу $P$ из своего варианта лабораторной работы номер 2 и все ее элементы умножить на $0.9$. В качестве вектора $f$ выбрать вектор $\pi$ из той же лабораторной работы. Если система получается несовместной или имеет не одно решение, то разрешается изменить матрицу А, домножив некоторые ее элементы на $-1$.

In [3]:
P = np.array([
    [0.75, 0, 0.25],
    [0.3, 0.5, 0.2],
    [0.2, 0.2, 0.6]
])
pi = np.array([
    [0.33333],
    [0.33333],
    [0.33334]
])

A = P*0.9
f = pi

N = 1000
m = 1000
ksi = np.zeros((m,3))

H = np.eye(3)

for l in range(m):
    print(f'{l}/{m}', end='\r')
    i = np.empty(N, dtype=int)
    Q = np.empty((N,3))

    i[0] = np.random.choice([0,1,2], p=pi.flatten())
    for k in range(1,N):
        i[k] = np.random.choice([0,1,2], p=P[i[k-1]])

    Q[0] = H[i[0]]/pi[i[0]] if pi[i[0]] > 0 else 0
    for k in range(1,N):
        Q[k] = (Q[k-1] * A[i[k-1],i[k]] / P[i[k-1],i[k]]) if P[i[k-1],i[k]] > 0 else 0

    for n in range(N):
        ksi[l] += Q[n]*f[i[n]]

x = np.mean(ksi, axis=0)
print('Approximate: ', x)
x_exact = np.linalg.solve(H-A, f)
print('Exact: ', x_exact.flatten())

Approximate:  [3.13002915 3.22003013 3.64993939]
Exact:  [3.33333216 3.33333099 3.33334645]
